In [56]:
# importando bibliotecas para todos os algoritmos

import pickle
import warnings
import numpy as np 
import pylab as pyl 
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn import ensemble
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

%pylab inline
%matplotlib inline 

Populating the interactive namespace from numpy and matplotlib


In [5]:
warnings.filterwarnings('ignore')

In [6]:
# importando o dataset
df = pd.read_csv('../dataset/data-set-clube-new-v2.csv', sep=';')

In [7]:
df.head(1)

,id_socio,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
0,38810,CANCELADO,24,0,0,0


In [8]:
df.tail(1)

,id_socio,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
3026,49083,ATIVO,51,4,12,15


In [9]:
# Removendo a coluna id_socio (não usaremos ela por enquanto, porém na etapa 3 ela será importante)
df.drop('id_socio', inplace=True, axis=1)

In [10]:
# Verificando se existem valores nulos
df.isnull().values.any()

False

Após o tratamento abaixo, o STATUS ficará: 

- ATIVO = 0 
- CANCELADO = 1

In [11]:
# Passando a coluna STATUS para o tipo inteiro
label_encoder = preprocessing.LabelEncoder()
df['status'] = label_encoder.fit_transform(df['status'])

In [12]:
# Definindo as classes
status_map = {True : 1, False : 0}

In [13]:
# Aplicando o mapeamento ao dataset
df['status'] = df['status'].map(status_map)

In [14]:
# conferindo a conversão da coluna
df.head(1)

,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
0,1,24,0,0,0


In [15]:
# conferindo a conversão da coluna
df.tail(1)

,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
3026,0,51,4,12,15


In [16]:
# Verificando como os dados estão distribuídos
num_true = len(df.loc[df['status'] == True])
num_false = len(df.loc[df['status'] == False])
print("Número de Status Cancelados: {0} ({1:2.2f}%)".format(num_true, (num_true/ (num_true + num_false)) * 100))
print("Número de Status Ativos    : {0} ({1:2.2f}%)".format(num_false, (num_false/ (num_true + num_false)) * 100))

Número de Status Cancelados: 771 (25.47%)
Número de Status Ativos    : 2256 (74.53%)


### Spliting

70% para dados de treino e 30% para dados de teste

In [17]:
# Seleção de variáveis preditoras (Feature Selection)
atributos = ['qtde_em_aberto', 'qtde_em_dia', 'qtde_em_atraso', 'qtde_frequencia_ano']

In [18]:
# Variável a ser prevista
atrib_prev = ['status']

In [19]:
# Criando objetos
X = df[atributos].values
Y = df[atrib_prev].values

In [20]:
# Definindo a taxa de split
split_test_size = 0.30

In [21]:
# Criando dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = split_test_size, random_state = 42)

In [22]:
# Imprimindo os resultados
print("{0:0.2f}% nos dados de treino".format((len(X_treino)/len(df.index)) * 100))
print("{0:0.2f}% nos dados de teste".format((len(X_teste)/len(df.index)) * 100))

69.97% nos dados de treino
30.03% nos dados de teste


### Verificando o Split

In [23]:
print("Original True : {0} ({1:0.2f}%)".format(len(df.loc[df['status'] == 1]), 
                                               (len(df.loc[df['status'] ==1])/len(df.index) * 100)))

print("Original False : {0} ({1:0.2f}%)".format(len(df.loc[df['status'] == 0]), 
                                               (len(df.loc[df['status'] == 0])/len(df.index) * 100)))
print("")
print("Training True : {0} ({1:0.2f}%)".format(len(Y_treino[Y_treino[:] == 1]), 
                                               (len(Y_treino[Y_treino[:] == 1])/len(Y_treino) * 100)))

print("Training False : {0} ({1:0.2f}%)".format(len(Y_treino[Y_treino[:] == 0]), 
                                               (len(Y_treino[Y_treino[:] == 0])/len(Y_treino) * 100)))
print("")
print("Test True : {0} ({1:0.2f}%)".format(len(Y_teste[Y_teste[:] == 1]), 
                                               (len(Y_teste[Y_teste[:] == 1])/len(Y_teste) * 100)))

print("Test False : {0} ({1:0.2f}%)".format(len(Y_teste[Y_teste[:] == 0]), 
                                               (len(Y_teste[Y_teste[:] == 0])/len(Y_teste) * 100)))

Original True : 771 (25.47%)
Original False : 2256 (74.53%)

Training True : 530 (25.02%)
Training False : 1588 (74.98%)

Test True : 241 (26.51%)
Test False : 668 (73.49%)


## 1 - Naive Bayes - GaussianNB

In [24]:
# Criando o modelo preditivo
modelo_GaussianNB = GaussianNB()

In [25]:
# Treinando o modelo
# ravel() "ajusta o shape" do treino pra treinar

modelo_GaussianNB.fit(X_treino, Y_treino.ravel())

GaussianNB(priors=None, var_smoothing=1e-09)

##### Verificando a exatidão no modelo nos dados de treino

In [26]:
# Fazendo as previsões somente com a base de teste e guardando em uma variável
gaussian_nb_predict_train = modelo_GaussianNB.predict(X_treino)

In [27]:
# Informa para o accuracy_score o Y_treino como o esperado, para fazer a comparação com a 
# variável acima gaussian_nb_predict_train
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_treino, gaussian_nb_predict_train)))
print()

# Abaixo está o resultado: a cada 100, está acertando 64

Exatidão (Accuracy): 0.6402



##### Verificando a exatidão no modelo nos dados de teste

In [28]:
gaussian_nb_predict_test = modelo_GaussianNB.predict(X_teste)

In [29]:
# Informa para o accuracy_score o Y_test como o esperado, para fazer a comparação com a variável
# acima nb_predict_test
# Diferente do modelo acima (nb_predict_train) esse bloco não conhece a base de treino (X_teste), então
# a previsão em teoria é mais assertiva.

print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, gaussian_nb_predict_test)))
print()

# Abaixo está o resultado: a cada 100, está acertando 64

Exatidão (Accuracy): 0.6469



In [30]:
# Criando uma Confusion Matrix
print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(Y_teste, gaussian_nb_predict_test, labels = [1, 0])))
print("")

print("Classification Report")
print(metrics.classification_report(Y_teste, gaussian_nb_predict_test, labels = [1, 0]))

Confusion Matrix
[[210  31]
 [290 378]]

Classification Report
              precision    recall  f1-score   support

           1       0.42      0.87      0.57       241
           0       0.92      0.57      0.70       668

   micro avg       0.65      0.65      0.65       909
   macro avg       0.67      0.72      0.63       909
weighted avg       0.79      0.65      0.67       909



## 2 - RandomForest

In [31]:
# Treinando o modelo
# ravel() "ajusta o shape" do treino pra treinar

modelo_RFClassifier = RandomForestClassifier(random_state = 42)
modelo_RFClassifier.fit(X_treino, Y_treino.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [32]:
# Verificando os dados de treino

# Boas chances de conseguir o Overfitting (resultados não muito precisos) com a base de treino, 
# pois o Algoritmo pode manter vícios e "lixos" nos resultados.
randomf_predict_train = modelo_RFClassifier.predict(X_treino)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_treino, randomf_predict_train)))

Exatidão (Accuracy): 0.9160


In [33]:
# Verificando nos dados de teste

# Esse bloco realmente pode ser levado em consideração
# Aqui apresentamos dados que  o algoritmo nunca viu
randomf_predict_test = modelo_RFClassifier.predict(X_teste)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, randomf_predict_test)))
print()

Exatidão (Accuracy): 0.8603



In [34]:
print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(Y_teste, randomf_predict_test, labels = [1, 0])))
print("")

print("Classification Report")
print(metrics.classification_report(Y_teste, randomf_predict_test, labels = [1, 0]))

# Abaixo a precisão está num total de 87%

Confusion Matrix
[[193  48]
 [ 79 589]]

Classification Report
              precision    recall  f1-score   support

           1       0.71      0.80      0.75       241
           0       0.92      0.88      0.90       668

   micro avg       0.86      0.86      0.86       909
   macro avg       0.82      0.84      0.83       909
weighted avg       0.87      0.86      0.86       909



## 3 - Regressão Logística

In [35]:
# Terceira versão do modelo usando Regressão Logística
modelo_regressaoLog = LogisticRegression(C = 0.7, random_state = 42)
modelo_regressaoLog.fit(X_treino, Y_treino.ravel())

# A avaliação deve ser feita SEMPRE na base de TESTE

regressao_log_predict_test = modelo_regressaoLog.predict(X_teste)

In [54]:
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, regressao_log_predict_test)))
print()
print("Classification Report")
print(metrics.classification_report(Y_teste, regressao_log_predict_test, labels = [1, 0]))

# Abaixo a precisão está num total de 83%

Exatidão (Accuracy): 0.8383

Classification Report
              precision    recall  f1-score   support

           1       0.69      0.72      0.70       241
           0       0.90      0.88      0.89       668

   micro avg       0.84      0.84      0.84       909
   macro avg       0.79      0.80      0.80       909
weighted avg       0.84      0.84      0.84       909



- Fazendo Previsões Com o Modelo Treinado Regressão Logística

In [47]:
# Salvando o modelo para posteriormente
filename = 'Saida_Previsao_Modelo_de_Treino_Regressao.sav'
pickle.dump(modelo_regressaoLog, open(filename, 'wb'))

Carregando o modelo e fazendo previsão com novos conjuntos de dados.

- X_teste, Y_teste devem ser novos conjuntos de dados preparados com o procedimento de limpeza e transformação adequados. Para o exemplo abaixo pegamos os índices 15 e 33 de X_teste.

In [67]:
loaded_model = pickle.load(open(filename, 'rb'))
resultado1 = loaded_model.predict(X_teste[15].reshape(1, -1))
resultado2 = loaded_model.predict(X_teste[33].reshape(1, -1))

print("Resultado de X_teste[15] para a previsão do status do sócio = {} (ATIVO)".format(resultado1))
print("Resultado de X_teste[33] para a previsão do status do sócio = {} (CANCELADO)".format(resultado2))

Resultado de X_teste[15] para a previsão do status do sócio = [0] (ATIVO)
Resultado de X_teste[33] para a previsão do status do sócio = [1] (CANCELADO)


## 4 - Gradient Boosting

In [83]:
modelo_gradientB_classifier = ensemble.GradientBoostingClassifier()
modelo_gradientB_classifier.fit(X_treino, Y_treino.ravel())

gradient_boosting_predict_test = modelo_gradientB_classifier.predict(X_teste)

print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, gradient_boosting_predict_test)))

Exatidão (Accuracy): 0.8581


## Comparação dos resultados dos Algoritmos (Acurácia X Tempo de Execução)

Foi levado em consideração SOMENTE os resultados dos testes, pois o resultado do treino não é tão confiável por seus resultados seguirem um "padrão vicioso".

#### 1 - Acurácia

In [84]:
# Porcentagem
porcentagem_nb_test = metrics.accuracy_score(Y_teste, gaussian_nb_predict_test)
porcentagem_rf_test = metrics.accuracy_score(Y_teste, randomf_predict_test)
porcentagem_rl_test = metrics.accuracy_score(Y_teste, regressao_log_predict_test)
porcentagem_gb_test = metrics.accuracy_score(Y_teste, gradient_boosting_predict_test)

#### 2 - Tempo de Execução. Medido pelo range(n)

- Usaremos um laço de execução acima de 10 mil. Um valor menor que esse resultou em resultados praticamente iguais para todos os algoritmos.

In [87]:
# Tempo de execução com RANGE 10.000 (Dez mil)
print("NAIVE BAYES: ")
%time for _ in range(10000): metrics.accuracy_score(Y_teste, gaussian_nb_predict_test)
    
print("RANDOM FOREST: ")
%time for _ in range(10000): metrics.accuracy_score(Y_teste, randomf_predict_test)

print("REGRESSÃO LINEAR: ")
%time for _ in range(10000): metrics.accuracy_score(Y_teste, regressao_log_predict_test)
    
print("GRADIENT BOOSTING")
%time for _ in range(10000): metrics.accuracy_score(Y_teste, gradient_boosting_predict_test)

NAIVE BAYES: 
CPU times: user 1.91 s, sys: 7.98 ms, total: 1.91 s
Wall time: 1.92 s
RANDOM FOREST: 
CPU times: user 1.88 s, sys: 24 ms, total: 1.9 s
Wall time: 1.9 s
REGRESSÃO LINEAR: 
CPU times: user 1.8 s, sys: 3.99 ms, total: 1.8 s
Wall time: 1.8 s
GRADIENT BOOSTING
CPU times: user 1.79 s, sys: 4.15 ms, total: 1.79 s
Wall time: 1.8 s


In [90]:
# Tempo de execução com RANGE 100.000 (Cem mil)
print("NAIVE BAYES: ")
%time for _ in range(100000): metrics.accuracy_score(Y_teste, gaussian_nb_predict_test)
    
print("RANDOM FOREST: ")
%time for _ in range(100000): metrics.accuracy_score(Y_teste, randomf_predict_test)

print("REGRESSÃO LINEAR: ")
%time for _ in range(100000): metrics.accuracy_score(Y_teste, regressao_log_predict_test)
    
print("GRADIENT BOOSTING")
%time for _ in range(100000): metrics.accuracy_score(Y_teste, gradient_boosting_predict_test)

NAIVE BAYES: 
CPU times: user 19.8 s, sys: 0 ns, total: 19.8 s
Wall time: 19.8 s
RANDOM FOREST: 
CPU times: user 20.4 s, sys: 7.95 ms, total: 20.4 s
Wall time: 20.4 s
REGRESSÃO LINEAR: 
CPU times: user 19.1 s, sys: 5 µs, total: 19.1 s
Wall time: 19.1 s
GRADIENT BOOSTING
CPU times: user 19 s, sys: 3.99 ms, total: 19 s
Wall time: 19 s


#### Populando o DataFrame Resultados

In [91]:
resultados = pd.DataFrame(
    {
        'Naive Bayes':[porcentagem_nb_test, '1.91 s', '19.8 s'],
        'Random Forest':[porcentagem_rf_test, '1.88 s', '20.4 s'],
        'Regressão Logística':[porcentagem_rl_test, '1.8 s', '19.1 s'],
        'Gradient Boosting':[porcentagem_gb_test, '1.79 s', '19 s'],
        '':['% de Acurácia', 'Time (Range 10000)', 'Time (Range 100000)']
    }
)

#### Tabela de Resultados

Abaixo podemos ver os resultados da acurácia e do tempo de execução de cada algoritmo.

- O algoritmo Gradient Boosting na maior parte dos testes foi o algoritmo que obteve o melhor tempo (nos dois padrões de tempo), e a sua acurácia quase se iguala a melhor acurácia dos algoritmos.

- O algoritmo Naive Bayes obteve uma acurácia abaixo da necessária (70%) ficando não só em quarto lugar entre os quatro algoritmos. O seu tempo de execução em boa parte dos resultados não ficou em uma boa classificação.

- Na execução exaustiva, o Random Forest foi o algoritmo que obteve a melhor acurácia entre todos os algoritmos testados, porém, o seu resultado de tempo de execução ficou em terceiro lugar boa parte dos testes. 

- O algoritmo Regressão Logística obteve uma boa acurácia se comparado com o Naive Bayes e seu tempo de execução foi próximo do Gradient Boosting.

- Houve uma necessidade de aumentar o tempo de teste para obter insights mais precisos nos resultados dos algoritmos, a diferença entre os resultados eram sempre menos de 1 segundo. Com a abordagem mais longa de teste os resultados passam de 1 segundo de diferente em alguns resultados.

In [92]:
resultados.head()

,,Gradient Boosting,Naive Bayes,Random Forest,Regressão Logística
0,% de Acurácia,0.858086,0.646865,0.860286,0.838284
1,Time (Range 10000),1.79 s,1.91 s,1.88 s,1.8 s
2,Time (Range 100000),19 s,19.8 s,20.4 s,19.1 s
